# 1. Introduction

## Main-Group Molecular Reactions  Benchmark Tests
The benchmark set of main-group molecular reactions[1] combines two recently published benchmark sets, created by Birkholz and Schlegel[2] and by Zimmerman[3]. The combined set includes 42 unique reactant configurations containing from 3 to 53 atoms. The systems contain elements H, B, C, N, O, F, Mg, P, S, Cl, and Br.

## Data Structure
The reactions, included in the benchmark set, are primarily single elementary step reactions but there are also a few multistep pathways, with two or more energy barriers along the RP. The reactant, product and saddle point configurations are accessible in a .xyz format.

## Calculation Details
All NEB calculations in this study are carried out using the NEB module implemented in the ORCA (4.2) suite of programs. All calculations use B3LYP+D3(BJ)/def2-SVP level of theory.

## References
1. J. Chem. Theory Comput. 2021, 17, 8, 4929–4945
2. J. Comput. Chem. 2015, 36, 1157– 1166
3. J. Chem. Theory Comput. 2013, 9, 7, 3043–3050

# 2. Run single-point calculations
The orginal data set can be found at './dataset', and only has coordinate information. We need to run single-point calculations to get energy and forces information. Orca files can be found at './singlepointcalc', and the calculation results can be found at './results'.

```python
python Run_Orca_Job.py
```

# 3. Data preprocessing
The data preprocessing is mainly to convert the data from 'json' format into 'mmap' & 'pkl' format.

In [5]:
import os
import json5
import numpy as np
import pickle as pkl

pkl_name_unique = np.array([f'system{i}' for i in range(1, 122)], dtype=np.str_)
pkl_name_idx = []

pkl_subset_unique = np.array(['react', 'sp', 'prod'], dtype=np.str_)
pkl_subset_idx = []

pkl_n_atoms = []

mmap_atomic_input = []
mmap_pos_idx_range = []
mmap_energies = []
mmap_forces = []

mmap_pos_idx_max = 0

for i in range(1, 122):
    for subset in ['react', 'sp', 'prod']:
        with open(f'./result/{i}_{subset}.json', 'r') as f:
            data = json5.load(f)
    name = f'system{i}'
    name_idx = np.where(pkl_name_unique == name)[0][0]
    pkl_name_idx.append(int(name_idx))
    subset_idx = np.where(pkl_subset_unique == subset)[0][0]
    pkl_subset_idx.append(int(subset_idx))
    pkl_n_atoms.append(len(data['atomic_number']))

    atomic_number = data['atomic_number']
    charge = data['charge']
    pos = data['positition']
    energy = data['energies']
    forces = data['forces']


    for atom_idx in range(len(atomic_number)):
        if len([atomic_number[atom_idx], charge[atom_idx], *pos[atom_idx]]) != 5:
            print(f'{name} {subset} {atom_idx}')
        mmap_atomic_input.append([atomic_number[atom_idx], charge[atom_idx], *pos[atom_idx]])

    mmap_pos_idx_range.extend([mmap_pos_idx_max, mmap_pos_idx_max + len(atomic_number)])
    mmap_pos_idx_max += len(atomic_number)
    mmap_energies.append(energy)
    mmap_forces.extend(forces)

pkl_dict = {
    'name': (pkl_name_unique, np.array(pkl_name_idx, dtype=np.int64)),
    'subset': (pkl_subset_unique, np.array(pkl_subset_idx, dtype=np.int64)),
    'n_atoms': np.array(pkl_n_atoms, dtype=np.int32),
}
# 写入pkl文件
pkl.dump(pkl_dict, open('dqc_cache/MGMR/preprocessed/props.pkl', 'wb'))


mmap_atomic_input = np.array(mmap_atomic_input, dtype=np.float32).reshape(-1)
mmap_pos_idx_range = np.array(mmap_pos_idx_range, dtype=np.int32).reshape(-1)
mmap_energies = np.array(mmap_energies, dtype=np.float64)
mmap_forces = np.array(mmap_forces, dtype=np.float32).reshape(-1)

# 写入mmap文件
mmap_ai_file = np.memmap('dqc_cache/MGMR/preprocessed/atomic_inputs.mmap', dtype=mmap_atomic_input.dtype, mode='w+', shape=mmap_atomic_input.shape)
mmap_ai_file[:] = mmap_atomic_input
mmap_ai_file.flush()

mmap_pi_file = np.memmap('dqc_cache/MGMR/preprocessed/position_idx_range.mmap', dtype=mmap_pos_idx_range.dtype, mode='w+', shape=mmap_pos_idx_range.shape)
mmap_pi_file[:] = mmap_pos_idx_range
mmap_pi_file.flush()

mmap_e_file = np.memmap('dqc_cache/MGMR/preprocessed/energies.mmap', dtype=mmap_energies.dtype, mode='w+', shape=mmap_energies.shape)
mmap_e_file[:] = mmap_energies
mmap_e_file.flush()

mmap_f_file = np.memmap('dqc_cache/MGMR/preprocessed/forces.mmap', dtype=mmap_forces.dtype, mode='w+', shape=mmap_forces.shape)
mmap_f_file[:] = mmap_forces
mmap_f_file.flush()

del mmap_ai_file, mmap_pi_file, mmap_e_file, mmap_f_file

# 4. MGMR Class

In [6]:
from openqdc.datasets.base import BaseDataset
from openqdc.methods import PotentialMethod

class MGMR(BaseDataset):
    """
    The benchmark set of main-group molecular reactions[1] combines two recently published benchmark sets, created by Birkholz and Schlegel[2] and by Zimmerman[3]. The combined set includes 42 unique reactant configurations containing from 3 to 53 atoms. The systems contain elements H, B, C, N, O, F, Mg, P, S, Cl, and Br.

    References
    1. J. Chem. Theory Comput. 2021, 17, 8, 4929–4945
    2. J. Comput. Chem. 2015, 36, 1157– 1166
    3. J. Chem. Theory Comput. 2013, 9, 7, 3043–3050
    """

    __name__ = "MGMR"

    __energy_methods__ = [
        PotentialMethod.B3LYP_D3_BJ_DEF2_TZVP
        # "wb97x/6-31G(d)",
    ]

    energy_target_names = [
        "B3LYP_D3_BJ_DEF2_TZVP.energy",
    ]

    __force_mask__ = [True]
    force_target_names = [
        "B3LYP_D3_BJ_DEF2_TZVP.forces",
    ]

    __energy_unit__ = "ev"
    __distance_unit__ = "ang"
    __forces_unit__ = "ev/ang"
    # __links__ = {"Transition1x.h5": "https://figshare.com/ndownloader/files/36035789"}

# 5. Usage

In [7]:
dataset = MGMR(
    energy_unit="ev",
    distance_unit="ang",
    array_format = "numpy",
    cache_dir='./dqc_cache',
    overwrite_local_cache=False
)

first_entry = dataset[0]
print(first_entry)
for data in dataset.as_iter(atoms=True):
    # data.edit()
    print(data) # Atoms object
    break

2025-09-10 16:08:25.160 | INFO     | openqdc.datasets.base:read_preprocess:430 - Reading preprocessed data.
2025-09-10 16:08:25.161 | INFO     | openqdc.datasets.base:read_preprocess:431 - Dataset MGMR with the following units:
                     Energy: ev,
                     Distance: ang,
                     Forces: ev/ang
2025-09-10 16:08:25.163 | INFO     | openqdc.datasets.base:read_preprocess:447 - Loaded atomic_inputs with shape (1798, 5), dtype float32
2025-09-10 16:08:25.164 | INFO     | openqdc.datasets.base:read_preprocess:447 - Loaded position_idx_range with shape (121, 2), dtype int32
2025-09-10 16:08:25.165 | INFO     | openqdc.datasets.base:read_preprocess:447 - Loaded energies with shape (121, 1), dtype float64
2025-09-10 16:08:25.166 | INFO     | openqdc.datasets.base:read_preprocess:447 - Loaded forces with shape (1798, 3, 1), dtype float32
2025-09-10 16:08:25.166 | INFO     | openqdc.datasets.base:read_preprocess:447 - Loaded subset with shape (121,), dtype <U5

{'positions': array([[ 0.67566854, -0.53062063, -0.4316012 ],
       [ 1.5674512 ,  0.7076675 ,  0.27234584],
       [-1.6351362 ,  0.30979663, -0.01891141],
       [-0.6214616 , -0.57879514,  0.30692858],
       [-1.5046947 ,  1.0907115 , -0.65368706],
       [ 1.0560939 ,  1.7389542 , -0.14566737],
       [ 2.7190435 ,  0.53905123, -0.10118867],
       [ 1.2115186 , -1.392715  , -0.3353055 ],
       [ 0.5983216 , -0.32296377, -1.4279795 ],
       [-2.4977899 ,  0.35396382,  0.50911635],
       [-0.7570233 , -1.4042009 ,  1.1695946 ],
       [ 1.4259484 ,  0.5658457 ,  1.4805013 ]], dtype=float32), 'atomic_numbers': array([7, 5, 7, 5, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32), 'charges': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32), 'e0': array([[-1485.85002872],
       [ -671.13951372],
       [-1485.85002872],
       [ -671.13951372],
       [  -13.66491465],
       [  -13.66491465],
       [  -13.66491465],
       [  -13.66491465],
       [  -13.66491465],
       [  -13.664

/home/wyk/Documents/code/openQDC/openqdc/utils/io.py:201: FutureWarning: Please use atoms.calc = calc
  at.set_calculator(Calculator())
